In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# import os
# os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async" # this is highly important for totally utilizing your GPU's memory, but it also breaks the profiler's memory breakdown

import numpy as np
import qutip as qt 
from bingo.optimizer.tf_adam_optimizer import AdamOptimizer
from bingo.gate_sets import SNAP
from bingo.optimizer.GateSynthesizer import GateSynthesizer
import matplotlib.pyplot as plt


Need tf version 2.3.0 or later. Using tensorflow version: 2.7.0



In [2]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
N = 40
# We initialize the ECDGateSet here
gate_set_params = {
    'N_cav' : N,
    'N_snap' : 10,
    'beta_scale' : 3.0
}
SNAP_gate_set = SNAP(**gate_set_params)

2022-09-16 18:49:20.360549: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1038] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-16 18:49:20.387431: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1038] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-16 18:49:20.387677: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1038] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-16 18:49:20.395889: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1038] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-16 18:49:20.396151: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1038] could n

In [3]:
#The target oscillator state.
fock = 4
psi_t = qt.basis(N,fock) #target state

In [4]:
#the optimization options
synth_params = {
    'gateset' : SNAP_gate_set,
    'N_blocks':3,
    'N_multistart' : 1000, #Batch size (number of circuit optimizations to run in parallel)
    'epochs' : 100, #number of epochs before termination
    'epoch_size' : 10, #number of adam steps per epoch
    'learning_rate' : 0.01, #adam learning rate
    'term_fid' : 0.995, #terminal fidelitiy
    'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
    'beta_scale' : 3.0, #maximum |beta| for random initialization
    'initial_states' : [qt.basis(N,0)], #qubit tensor oscillator, start in |g> |0>
    'target_states' : [psi_t], #end in |e> |target>.
    'name' : 'SNAP Fock %d' % fock, #name for printing and saving
    'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}
gatesynth = GateSynthesizer(**synth_params)

In [5]:
#create optimization object. 
#initial params will be randomized upon creation
opt = AdamOptimizer(gatesynth)

#print optimization info. this lives in gatesynth, since we eventually want to fully abstract away the optimizer
gatesynth.print_info()

N_blocks: 3
optimization_type: state transfer
optimization_masks: {'betas_rho': None, 'betas_angle': None, 'thetas': None}
target_unitary: None
expectation_operators: None
target_expectation_values: None
N_multistart: 1000
term_fid: 0.995
dfid_stop: 1e-06
learning_rate: 0.01
epoch_size: 10
epochs: 100
name: SNAP Fock 4
filename: None
comment: 
coherent: False
timestamps: []
do_prints: True
beta_scale: 3.0
filename: SNAP Fock 4.h5

Best circuit parameters found:
betas:    tf.Tensor([ 0.16757-0.20361j  0.08439-1.56395j -0.63772-0.27193j], shape=(3,), dtype=complex64)
thetas:    tf.Tensor(
[[-3.05451 -2.77744  2.81009 -0.74968  0.5672  -2.67061 -3.10376 -0.63526
  -2.89279  1.92504]
 [ 1.03088  0.52305  0.5322   3.09841  2.77599 -0.25723  1.4873   0.53802
   1.68362  1.99334]
 [ 0.17808  2.45483  0.90632  0.0519   1.76652  1.58502  2.56119 -1.67299
   2.89432 -1.59513]], shape=(3, 10), dtype=float32)

 Best circuit Fidelity: 0.589094




In [6]:
#run optimizer.
#note the optimizer can be stopped at any time by interrupting the python consle,
#and the optimization results will still be saved and part of the opt object.
#This allows you to stop the optimization whenever you want and still use the result.
opt.optimize()

Start time: 2022-09-16 18:52:15
 Epoch: 11 / 100 Max Fid: 0.995921 Avg Fid: 0.691235 Max dFid: 0.129073 Avg dFid: 0.023869 Elapsed time: 0:00:02.408573 Expected remaining time: 0:00:19.487546 (real part)

 Optimization stopped. Term fidelity reached.

N_blocks: 3
optimization_type: state transfer
optimization_masks: {'betas_rho': None, 'betas_angle': None, 'thetas': None}
target_unitary: None
expectation_operators: None
target_expectation_values: None
N_multistart: 1000
term_fid: 0.995
dfid_stop: 1e-06
learning_rate: 0.01
epoch_size: 10
epochs: 100
name: SNAP Fock 4
filename: None
comment: 
coherent: False
timestamps: ['2022-09-16 18:52:15']
do_prints: True
beta_scale: 3.0
filename: SNAP Fock 4.h5

Best circuit parameters found:
betas:    tf.Tensor([ 1.39335-0.89213j -0.64134-0.82991j -0.28165+0.51639j], shape=(3,), dtype=complex64)
thetas:    tf.Tensor(
[[ 1.82228 -1.13672 -2.70646 -1.52249  1.49545 -0.37888 -1.53095 -2.16632
  -1.69706  2.47704]
 [-1.62892 -2.70621 -2.85576  2.1404  

'2022-09-16 18:52:15'

In [7]:
#can print info, including the best circuit found.
gatesynth.print_info() 

N_blocks: 3
optimization_type: state transfer
optimization_masks: {'betas_rho': None, 'betas_angle': None, 'thetas': None}
target_unitary: None
expectation_operators: None
target_expectation_values: None
N_multistart: 1000
term_fid: 0.995
dfid_stop: 1e-06
learning_rate: 0.01
epoch_size: 10
epochs: 100
name: SNAP Fock 4
filename: None
comment: 
coherent: False
timestamps: ['2022-09-16 18:52:15']
do_prints: True
beta_scale: 3.0
filename: SNAP Fock 4.h5

Best circuit parameters found:
betas:    tf.Tensor([ 1.39335-0.89213j -0.64134-0.82991j -0.28165+0.51639j], shape=(3,), dtype=complex64)
thetas:    tf.Tensor(
[[ 1.82228 -1.13672 -2.70646 -1.52249  1.49545 -0.37888 -1.53095 -2.16632
  -1.69706  2.47704]
 [-1.62892 -2.70621 -2.85576  2.1404  -1.58777  1.59087 -2.01933  0.69693
   3.00707 -1.28405]
 [ 1.12914 -2.38587  0.0219  -1.98821 -1.30007  1.26505  2.7885  -1.78838
   0.04691  2.00402]], shape=(3, 10), dtype=float32)

 Best circuit Fidelity: 0.996872




In [ ]:
print(gatesynth.best_fidelity())

In [ ]:
#can also get the best circuit parameters directly, could save this to a .npz file.
best_circuit =  gatesynth.best_circuit()
betas = best_circuit['betas']
thetas = best_circuit['thetas']
print(best_circuit)

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
# Launch TensorBoard and navigate to the Profile tab to view performance profile
%tensorboard --logdir=logs